In [ ]:
import time
import numpy as np
from functools import wraps

class Platform:
    def __init__(self, config_name, table_path):
        self.table = tables # get table...

        config = config.ConfigParser()
        config.read(config_name)


        self.algo_instance = config.get('plug', 'AlgorithmClass')
        self.preprocessing_type = config.get('plug', 'PreprocessingClass')
        self.run_type = config.get('plug', 'runType')
        self.logs = config.get('plug', 'logs')

        match self.run_type:
          case 'test':
            self.test(algo, table, logs)
          case 'compress':
            self.compress(algo, table, logs)
          case 'decompress':
            self.compress(algo, table, logs)

    def measure_time(self, log=False):
        def decorator(func):
            def wrapper(*args, **kwargs):
                start_time = time.time()
                result = func(*args, **kwargs)
                end_time = time.time()
                if log:
                    print(f"Function {func.__name__} took {end_time - start_time} seconds to execute.")
                return result
            return wrapper
        return decorator


    @measure_time(log=self.logs)
    def compress(self, algo, table):
          pass

    def decompress(self):
          pass

    def test(self, algo, table):
        start_time = time.time()
        compressed_data = algo.compress(np.array(table))
        end_time = time.time()
        compression_rate = len(compressed_data) / len(table)
        decompressed_data = algo.decompression(compressed_data)
        loss_rate = (table - decompressed_data) / table
        result = {
            "algorithm": algo.__name__,
            "table": table,
            "compression_rate": compression_rate,
            "time_taken_to_compress": end_time - start_time,
            "loss_rate": loss_rate
        }
        self.results.append(result)

In [1]:
print("hello, world")

hello, world
